<a href="https://colab.research.google.com/github/amkuipers/colab_scripts/blob/main/RSA_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RSA encryption / decryption

Theory and step by step example.

# RSA Theory

You should use tested libraries for proper RSA. Since the theory below is my interpretation of other pages explaining it to me. And I am not a mathematician.

##  RSA Keys

State a bit size of $N$. This is a power of 2.
$$bitsize(N) = 1024$$
Then select random primes $p$ and $q$ so that the $bitsize(N)$ has that value.

$$N=pq$$

Determine $phi$
  $$\phi = (p-1)(q-1)$$
Pick $e$
$$e=3..65537$$
Compute $d$
$$d=e^{-1} \pmod\phi$$
This gives public key $$k_{public}=(N,e)$$
And private key $$k_{private}=(N,d)$$
Keep the private key, well ... private, and $p$, $q$ and $phi$ too.

## RSA Encryption and Decryption

Have secret message $m$ represented as an integer number. Where
$$m \in 1..(N-1)$$
If it doesn't fit, then break $m$ in multiple values, blocks. Then encrypt each block.

Encrypt $m$ with public key $(N,e)$ to ciphertext $c$ as
$$c=m^e\pmod{N}$$

Decrypt $c$ with private key $(N,d)$ to $m$ as
$$m=c^d\pmod{N}$$

## RSA proof

See other resources that this concept works.

# RSA CTF
In _capture the flag_ (CTF) challenges, there is a _crypto_ category that can use RSA.

## RSA e=3 and small message weakness
When $e=3$ and the $m$ was a small number so that $m^e < N$ then $c$ can be deciphered to $m$
  $$m=\sqrt[3]{c}$$ 
because there was no $\pmod{N}$ involved. Of course $m$ is unknown, so just try it, and if $m$ results in an integer value that is the answer.

## RSA e=3 weakness

When $e=3$ and the $m^e > N$ then $c$ can be deciphered using iterations of $i*N$, since there was a $\pmod{N}$ involved here.
  $$m=\sqrt[e]{c+iN}$$

While iterating $i$, $m$ results in a floating number. In case it is (close to) an integer, then it leads to the answer, so have that in place; example $41.999999998$ is $42$. This requires some brute force, and select a proper range for $i$.

## RSA small N weakness

If $N$ is a relative small value, then it might be possible to determine primes $p$ and $q$.
$$N=pq$$
In case $p$ and $q$ are the same, this means that 
$$p=q=\sqrt{N}$$
But this situation is not to be expected, $p$ is less and $q$ is more than this square root of $n$.

So if we iterate prime numbers up to $\sqrt{N}$, then $p$ is a prime of that series, and $q=N/p$. In case $q$ calculates to an integer, then the $p$ and $q$ are found.

If the primes are known, then the public key and the private key can be calculated. And so all is known.


In [1]:
import math

# Step 1: Choose two random primes, and calculate modulus
In example:
- This example uses two small primes, as input for calculations, to demonstrate the algorithm. 

In reality:
- A key size, the number of bits of n, is the starting point, that determines also the bit size of its factors
- The primes have the same bit size.
- A proper set of primes have hundreds of digits. 
- So that n, the product of the two primes, that is in the public key, cannot be calculated back.
- The primes should not public.
- A proper random number generator is used to generated the two primes.


Product n is part of the public key, the primes are kept secret. N is also called modulus, since it is used to do modulus calculations.





In [2]:
p = 19  
q = 29  

n = p*q


In example:
- Value n is not safe

In reality:
- The value of n is safe when it has > 512 bits

Algorithm choice:
- Algoritm 1 = `math.floor(math.log2(n)) + 1`
- Algoritm 2 = `int.bit_length(n)`  

In [3]:
print(f'N = {n} that has key length of {int.bit_length(n)} bits')

if int.bit_length(n) % 8 > 0:
  print(f'WARN: Key size is not aligned to 8 bits, block size impacted')
else:
  print(f'INFO: Key size is aligned to 8 bits')

if int.bit_length(p) != int.bit_length(q):
  print(f'WARN: The two primes do not have the same bit size')
else:
  print(f'INFO: The two primes have the same bit size')

bits = int.bit_length(n)
if bits < 512:
  print(f'WARN: The value of n = {n} is not safe, since {bits} bits is less than 512 bits')
else:
  print(f'INFO: The value of n = {n} is safe, since {bits} bits is more than 512 bits')

N = 551 that has key length of 10 bits
WARN: Key size is not aligned to 8 bits, block size impacted
INFO: The two primes have the same bit size
WARN: The value of n = 551 is not safe, since 10 bits is less than 512 bits


# Step 2: Calculate ϕ(n)

$$\phi=\phi(N)=phi(N)=(p-1)(q-1)$$

- This is called Euler's totient. 
And it is a calculation step.

My notes:
- ϕ is less than N
- My assumption is that the (p-1) and the (q-1) are probably not primes, so can have multiple divisors. 
- This page shows ϕ as different symbol in the Code block as in the Text block, but it is the same.

In [4]:
phi = (p-1)*(q-1)


Log

In [5]:
print(f"ϕ(n) = {phi}")

ϕ(n) = 504


# Step 3: Choose e

Choose `e` so `gcd(e,ϕ(n)) = 1`. 
- This means that `e` and `ϕ(n)` do not have common factors.
- `gcd(a,b)` is great common divider; what is the largest whole value x that can be found, that is shared by a and by b. So `a/x` and `b/x` both result in whole integers.
- the `gcd(e,ϕ(n))` must be `1`, so that it is proven that there are no common factors. 

Note:
- In practice e is between 3 and 65537. 
- An e = 3 is weak?
- Reason that 3, 7 and 65537 are often seen is that in binary it has only two '1' bits. And every '1' bit cost calculation time. A '0' bit does not, in an efficient calculation.
So that even smartcards can also calculate RSA, when there are less bits a '1'.

In [6]:
# examples with two '1' bits
for i in range(1, 16+1):
  b = 2**i + 1
  print(f'Example: Bits of {b} are {bin(b)}')

Example: Bits of 3 are 0b11
Example: Bits of 5 are 0b101
Example: Bits of 9 are 0b1001
Example: Bits of 17 are 0b10001
Example: Bits of 33 are 0b100001
Example: Bits of 65 are 0b1000001
Example: Bits of 129 are 0b10000001
Example: Bits of 257 are 0b100000001
Example: Bits of 513 are 0b1000000001
Example: Bits of 1025 are 0b10000000001
Example: Bits of 2049 are 0b100000000001
Example: Bits of 4097 are 0b1000000000001
Example: Bits of 8193 are 0b10000000000001
Example: Bits of 16385 are 0b100000000000001
Example: Bits of 32769 are 0b1000000000000001
Example: Bits of 65537 are 0b10000000000000001


So e:
- Small exponent e.
- Choose a value for e in the range of 1..phi(n)

- Found this text, but I don't understand it yet: "Is a relatively coprime with (p-1) and (q-1)". And a Coprime = When two numbers have no common factors other than 1. So the value is not a real prime.
- This means; find e that the greatest common divisor of phi and e is 1.

Algorithm:
- Loop e and 
- check gcd( phi, e) == 1 and
- e has two '1' bits, 
- so that the resulting list of e candidates is smaller.

In [7]:
print(f'For e, pick a value between {1} and {phi}')

for e in range(phi-1, 1+1, -1):
  gc = math.gcd( phi, e)
  if gc == 1:
    # found GCD 
    
    ones_count = str(bin(e)).count('1')
    if ones_count == 2:
      # found two '1' bits in e
      print(f'Found GCD for phi,e with e={e} and has two bits set to 1')

      # 65 is also an answer, it is not a prime, should e be a prime?

# manually selected
e = 17
print(f'Selected e = {e}')

For e, pick a value between 1 and 504
Found GCD for phi,e with e=257 and has two bits set to 1
Found GCD for phi,e with e=65 and has two bits set to 1
Found GCD for phi,e with e=17 and has two bits set to 1
Found GCD for phi,e with e=5 and has two bits set to 1
Selected e = 17


# Step 4: Calculate d

The $d$ is part of the secret key.

$$d = e^{−1} \pmod {\phi(N)} $$

The private key has secret exponent d.

- Found text, but I need to look into this: ed == 1(mod phi). Where == is congruent?

Note on algorithm:
- found this code
- would like to refactor and remove the itertools

In [8]:
import itertools

def get_d(e, phi):
  for i in itertools.count(start=int(phi / e)):
    v = ( e * i ) % phi
    # print(f"debug {i} and {v}")
    if v == 1:
      break
  return i  # d

d = get_d(e, phi)
print(f'Secret d is {d}')

Secret d is 89


# Step 5: Public and Private keys

The Public Key is (n, e)

The Private Key is (n, d)

In [9]:
pub = (n, e)
prv = (n, d)

print(f"Public key {pub}")
print(f"Private key {prv}")


Public key (551, 17)
Private key (551, 89)


# Step 6: The message
The message to be encrypted.

In the example:
- Original message is represented as integer array, with ASCII values of the characters.
- Use the `ord()` value 8 bits as the message block.
- The message is "HELP!".

In reality:
- Use a message block as long as the key length.
- Use padding algorithm to pad the last message block, if needed. 

In [10]:
m = [ord(ch) for ch in "HELP!"]
print(m)

[72, 69, 76, 80, 33]


# Step 7: Encrypt the message
Encrypt the message with public key, using n, e.

Resulting in ciphertext c.

Note on the implementation:
- Python `math.pow(m, e)` is floating point and does no modulo n.
- Python inbuilt function `pow(m, e, n)` already does the modulo n.
- Plain `m**e%n` is an integer calculation that is working but slow

In [11]:
# c = [i**e % n for i in m]
# print(f'Encrypted ciphertext {c}')

c = [pow(i,e,n) for i in m]
print(f'Encrypted ciphertext {c}')

Encrypted ciphertext [185, 293, 171, 5, 528]


## Notice values!
The values from this are modulo N, so it means that these can be larger than 256. So it requires more bits/bytes to store.

# Step 8: Decrypt the message
Decrypt the ciphertext c back to a message using the private key (n, d).

In [12]:
# mm = [i**d % n for i in c]
mm = [pow(i,d, n) for i in c]

print(f'Decrypted {mm}')
print("Decrypted message", ''.join([chr(i) for i in mm]))

Decrypted [72, 69, 76, 80, 33]
Decrypted message HELP!


# Step 9: Crack the public key

Now attempt to 'crack' the public key; factoring out the two primes of the product n. Then from these primes the private key can be calculated. So that messages can be encrypted / decrypted / signed / verified.

Notes:
- This is for small values of n only. 

Steps:
- We know that n is the product of only the two primes
- We know that 1 of the primes is less than the square root of n
- Iterate from value 3 in steps of 2 to avoid the even value calculations, since these are not primes anyway. Although it still iterates through multiples of other primes, like 15.
- If a division results in a remainder of 0, then we found both primes

Improvement steps:
- Iterate from sqrt(n) down instead of from 3 up. This is because the p and q have an equal bit size, so it is likely to be closer to the sqrt(n) than to 3.

In [13]:
def crack_n(n):
  # find the two primes, so that p2*q2 = n
  sq = int(math.sqrt(n))
  #print(f'Looking for primes, with p up to sqrt(n): sqrt({n}) = {sq}')
  for p2 in range(3, sq+1, 2): 
    if n % p2 == 0:
      return p2, n//p2 # returns p, q

#primes = crack_n(n)     
#print(f"Primes: {primes}")

def crack_pub(pub):
  n2, e2 = pub
  p2, q2 = crack_n(n)
  phi2 = (p2-1)*(q2-1)
  d2 = get_d(e2, phi2)
  return n2, d2 # returns private key (n, d)

print(f'Public key {pub} cracked to private key {crack_pub(pub)}')

Public key (551, 17) cracked to private key (551, 89)


# Appendix
Code and comments moved out of the calculaton steps above. Kept before deletion or before improvement to move back into the calculation steps.

## Alternative d calculation
Alternative d calculation;

- Euclidean algoritm = gcd() = greatest common divider
- Extended Euclidean algoritm = coefficients for Bezouts identity; one is d

https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm

Euclidean algorithm = modular inverse of e(mod n)

In [14]:
# incorrect, provide phi for parameter n
# so this must be looked into
# because for phi you already need to know the p,q factors of n

def inverse(a, n):
  """ Euclidean algorithm """
  print(f"inverse({a},{n})")
  t, newt = 0, 1
  r, newr = n, a

  while newr:
    quotient = r // newr  # floor division
    t, newt = newt, t - quotient * newt
    r, newr = newr, r - quotient * newr
    
  if r > 1:
    return None  # there's no solution
    
  if t < 0:
    t = t + n
    
  return t

# this is the same input and output as in the original d calculation
d2 = inverse(e, phi )
print(f"d2 = {d2}")




inverse(17,504)
d2 = 89


## Appendix: Using RSA library
The rsa library contains proper tested implementations. 

Here it prints out values to follow along.

In [15]:
# included output to follow along

import rsa
import binascii

plaintext = input("Enter your plaintext message:")
print(f'Generating key pair and encrypting the message...')

# the bit length of N that is required
KEY_LENGTH = 2048

publickey, privatekey = rsa.newkeys(KEY_LENGTH)
print(f'Public key  <-- share')
#print(f'Public key  : {publickey}')
print(f'          e : {publickey.e}')
print(f'          n : {publickey.n}')
print(f'        PEM :\n{publickey.save_pkcs1("PEM").decode()}')

print(f'Private key <-- never share')
#print(f'Private key : {privatekey}')
print(f'          e : {privatekey.e}')
print(f'          n : {privatekey.n}')
print(f'          d : {privatekey.d}')
print(f'          p : {privatekey.p}')
print(f'          q : {privatekey.q}')
print(f'        PEM :\n{privatekey.save_pkcs1("PEM").decode()}')

#print(f'Message     : {plaintext}')

ciphertext = rsa.encrypt(plaintext.encode(), publickey)
#print(f'Ciphertext  : {ciphertext}')
print(f'Ciphertext  : {binascii.hexlify(ciphertext)}')

decrypted_text = rsa.decrypt(ciphertext, privatekey).decode()
print(f'Decrypted   : {decrypted_text}')


Enter your plaintext message:Hello, World!
Generating key pair and encrypting the message...
Public key  <-- share
          e : 65537
          n : 28265342317026481885788508253905339824219365363031595692215807635842707616078096257430791686884293933587987502388940531857531127445638950079776758875007353051184889881646853402069084911483113068755501334873075460375367419653342627426020062897106362867251714187396806616591097759349587953117588620274153189506097455240773427517497093149128728039000045383353518950737051885966931737555900943932122880718793165487579256306738070433581810723975451962318868187607307057594901147759355886558703911970480415154689130249522246624714536207325683175367353710846414290643002166256664273024131357075599236302957663981415198637703
        PEM :
-----BEGIN RSA PUBLIC KEY-----
MIIBCgKCAQEA3+eWiI/5I7m9ZDsi314Iw11b6wspA0JRJREseOqmGXtBM3zktiYW
2ExdeYzrPVWPHN9rAoSYvD74+uZ1zeve27T3D+qYX19gL0rmmuvNFrymXhWOMq3U
fq/+dlny+g8GACBhY6tEfzMFv52yIRHLzHBkTzryUECsoRkP6LGZFrPlZX

## Appendix: Interactive

Requires e,n and d to be set.

Note:
- because in the example n is small, every m character in the input is encrypted. 
- Every m character is encrypted with the same result. And this is a problem.
- In reality n is 1024 bits minimum. So 128 bytes. There are rules for the size of the message and the size of n, but not yet here. I assume that from m, a block of bits is taken to be encrypted, where the block size is the number of bits of n. When the remaining message bits is less then the number of bits in n, then m is padded with certain padding algorithms.

Example:
- When typing `Helloooooo` the `o` is also repeating in the encrypted message. That is not good, since text frequency analysis can be used on long enough messages.

In [16]:
message = input("Enter the text message to be encrypted: ")
m = [ord(ch) for ch in message]

# encryption using public key n,e
c = [pow(i, e, n) for i in m]
print("Ciphertext is:", c)

# decryption using private key n,d
mm = [pow(i, d, n) for i in c]
print(f'Decrypted: {mm}')
print("Decrypted message:", ''.join([chr(i) for i in mm]))



Enter the text message to be encrypted: Hello, World!
Ciphertext is: [185, 301, 193, 193, 310, 453, 60, 406, 310, 95, 193, 80, 528]
Decrypted: [72, 101, 108, 108, 111, 44, 32, 87, 111, 114, 108, 100, 33]
Decrypted message: Hello, World!


# Some of the used references
- http://www.jtrive.com/simple-example-of-rsa-encryption-and-decryption.html
- https://www.di-mgt.com.au/rsa_alg.html
- https://medium.com/asecuritysite-when-bob-met-alice/in-rsa-we-have-e-d-n-p-and-q-but-what-are-dq-dp-and-invq-79a85fff605a


